In [1]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

Vamos a trabajar con datos que describen el comportamiento de los clientes dentro de un programa de lealtad de una aerolínea:
- Customer Flight Analysis.csv
- Customer Loyalty History.csv

--> CUSTOMER FLIGHT ACTIVITY

In [17]:
df_cfa = pd.read_csv(r'data\Customer Flight Activity.csv')
display(df_cfa.head(3)) # miramos cómo nos abre el df
display(df_cfa.tail(3))
print('Valores aleatorios:')
display(df_cfa.sample(3))

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
0,100018,2017,1,3,0,3,1521,152.0,0,0
1,100102,2017,1,10,4,14,2030,203.0,0,0
2,100140,2017,1,6,0,6,1200,120.0,0,0


,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
405621,999940,2018,12,3,0,3,1233,123.0,0,0
405622,999982,2018,12,0,0,0,0,0.0,0,0
405623,999986,2018,12,0,0,0,0,0.0,0,0


Valores aleatorios:


,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
171808,250711,2017,11,0,0,0,0,0.0,0,0
308705,152681,2018,10,13,2,15,1920,192.0,553,45
34197,172259,2018,5,0,0,0,0,0.0,0,0


En este df, tenemos la siguiente información:
- `Loyalty Number`: identificador único para cada cliente dentro del programa de lealtad de la aerolínea. 
- `Year`: año en el cual se registraron las actividades de vuelo para el cliente.
- `Month`: mes del año (de 1 a 12) en el cual ocurrieron las actividades de vuelo.
- `Flights Booked`: número total de vuelos reservados por el cliente en <u>ese mes específico</u>.
- `Flights with Companions`: número de vuelos reservados en los cuales el cliente viajó con acompañantes. 
- `Total Flights`: número total de vuelos que el cliente ha realizado (puede incluir vuelos reservado en meses anteriores).
- `Distance`: distancia total (presumiblemente en millas o kilómetros) que el cliente ha volado <u>durante el mes</u>.
- `Points Accumulated`: puntos acumulados por el cliente en el programa de lealtad <u>durante el mes</u>,
con base en la distancia volada u otros factores.
- `Points Redeemed`: puntos que el cliente ha redimido <u>en el mes</u>, posiblemente para obtener beneficios como vuelos gratis, mejoras, etc.
- `Dollar Cost Points Redeemed`: valor en <u>dólares</u> de los puntos que el cliente ha redimido <u>durante el mes</u>.

In [18]:
# VISUALIZACIÓN INICIAL
df_cfa.info()
print(f'--> Tenemos {df_cfa.shape[0]} filas y {df_cfa.shape[1]} columnas.')

# NOTAS:
# nombres col separadas por espacios --> guión bajo para evitar errores
# parece que no hay ningún nulo en ninguna columna
# SON TODAS NUMÉRICAS
# Dollar Cost Points Redeemed es int --> mejor lo ponemos en FLOAT (valor en dólares)
# num de clientes = 405624

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405624 entries, 0 to 405623
Data columns (total 10 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Loyalty Number               405624 non-null  int64  
 1   Year                         405624 non-null  int64  
 2   Month                        405624 non-null  int64  
 3   Flights Booked               405624 non-null  int64  
 4   Flights with Companions      405624 non-null  int64  
 5   Total Flights                405624 non-null  int64  
 6   Distance                     405624 non-null  int64  
 7   Points Accumulated           405624 non-null  float64
 8   Points Redeemed              405624 non-null  int64  
 9   Dollar Cost Points Redeemed  405624 non-null  int64  
dtypes: float64(1), int64(9)
memory usage: 30.9 MB
--> Tenemos 405624 filas y 10 columnas.


--> CUSTOMER LOYALTY HISTORY

In [15]:
df_clh = pd.read_csv(r'data\Customer Loyalty History.csv')
display(df_clh.head(3)) # miramos cómo nos abre el df
display(df_clh.tail(3))
print('Valores aleatorios:')
display(df_clh.sample(3))

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
0,480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN
1,549612,Canada,Alberta,Edmonton,T3G 6Y6,Male,College,NaN,Divorced,Star,3839.61,Standard,2016,3,NaN,NaN
2,429460,Canada,British Columbia,Vancouver,V6E 3D9,Male,College,NaN,Single,Star,3839.75,Standard,2014,7,2018.0,1.0


,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
16734,776187,Canada,British Columbia,Vancouver,V5R 1W3,Male,College,NaN,Single,Star,74228.52,Standard,2014,3,NaN,NaN
16735,906428,Canada,Yukon,Whitehorse,Y2K 6R0,Male,Bachelor,-57297.0,Married,Star,10018.66,2018 Promotion,2018,4,NaN,NaN
16736,652627,Canada,Manitoba,Winnipeg,R2C 0M5,Female,Bachelor,75049.0,Married,Star,83325.38,Standard,2015,12,2016.0,8.0


Valores aleatorios:


,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
14598,234836,Canada,Quebec,Hull,J8Y 3Z5,Female,Bachelor,83322.0,Married,Star,7462.93,Standard,2017,4,2017.0,12.0
4550,417904,Canada,Manitoba,Winnipeg,R2C 0M5,Male,College,NaN,Single,Nova,2912.24,Standard,2017,1,NaN,NaN
1038,378181,Canada,British Columbia,Vancouver,V6E 3Z3,Female,Bachelor,77329.0,Divorced,Aurora,5278.20,Standard,2014,3,NaN,NaN


En este df tenemos la siguiente información:
- `Loyalty Number`: identificador único del cliente dentro del programa de lealtad.
- `Country`: país de residencia del cliente.
- `Province`: provincia o estado de residencia del cliente (aplicable a países con divisiones provinciales o estatales, como Canadá).
- `City`: ciudad de residencia del cliente.
- `Postal Code`: código postal del cliente.
- `Gender`: género del cliente.
- `Education`: nivel educativo alcanzado por el cliente.
- `Salary`: ingreso anual estimado del cliente.
- `Marital Status`: estado civil del cliente.
- `Loyalty Card`: tipo de tarjeta de lealtad que posee el cliente (distintos niveles o
categorías dentro del programa de lealtad)
- `CLV (Customer Lifetime Value)`: valor total estimado que el cliente aporta a la empresa durante toda la relación que mantiene con ella.
- `Enrollment Type`: tipo de inscripción del cliente en el programa de lealtad.
- `Enrollment Year`: año en que el cliente se inscribió en el programa de lealtad.
- `Enrollment Month`: mes en que el cliente se inscribió en el programa de lealtad.
- `Cancellation Year`: año en que el cliente canceló su membresía en el programa de lealtad, si aplica.
- `Cancellation Month`: mes en que el cliente canceló su membresía en el programa de lealtad, si aplica.

In [16]:
# VISUALIZACIÓN INICIAL
df_clh.info()
print(f'--> Tenemos {df_clh.shape[0]} filas y {df_clh.shape[1]} columnas.')

# NOTAS:
# nombres col separadas por espacios --> guión bajo para evitar errores
# NULOS: Salary,  Cancellation Year , Cancellation Month 
# Cancellation year // Cancellation month --> a int
# num de clientes (cfa) = 405624    <-->    16737

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16737 entries, 0 to 16736
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Loyalty Number      16737 non-null  int64  
 1   Country             16737 non-null  object 
 2   Province            16737 non-null  object 
 3   City                16737 non-null  object 
 4   Postal Code         16737 non-null  object 
 5   Gender              16737 non-null  object 
 6   Education           16737 non-null  object 
 7   Salary              12499 non-null  float64
 8   Marital Status      16737 non-null  object 
 9   Loyalty Card        16737 non-null  object 
 10  CLV                 16737 non-null  float64
 11  Enrollment Type     16737 non-null  object 
 12  Enrollment Year     16737 non-null  int64  
 13  Enrollment Month    16737 non-null  int64  
 14  Cancellation Year   2067 non-null   float64
 15  Cancellation Month  2067 non-null   float64
dtypes: f

### Análisis Descriptivo

--> CUSTOMER FLIGHT ACTIVITY

In [ ]:
print("Estadísticas descriptivas para VARIABLES NUMÉRICAS:")
# Loyalty Number / 
display(df_cfa.describe().T)
print("\nEstadísticas descriptivas para VARIABLES CATEGÓRICAS:")
print("--> No tenemos variables categóricas en este df.")

# NOTAS:


Estadísticas descriptivas para VARIABLES NUMÉRICAS:


,count,mean,std,min,25%,50%,75%,max
Loyalty Number,405624.0,550037.873084,258935.286969,100018.0,326961.00,550834.0,772194.00,999986.0
Year,405624.0,2017.500000,0.500001,2017.0,2017.00,2017.5,2018.00,2018.0
Month,405624.0,6.500000,3.452057,1.0,3.75,6.5,9.25,12.0
Flights Booked,405624.0,4.115052,5.225518,0.0,0.00,1.0,8.00,21.0
Flights with Companions,405624.0,1.031805,2.076869,0.0,0.00,0.0,1.00,11.0
Total Flights,405624.0,5.146858,6.521227,0.0,0.00,1.0,10.00,32.0
Distance,405624.0,1208.880059,1433.155320,0.0,0.00,488.0,2336.00,6293.0
Points Accumulated,405624.0,123.692721,146.599831,0.0,0.00,50.0,239.00,676.5
Points Redeemed,405624.0,30.696872,125.486049,0.0,0.00,0.0,0.00,876.0
Dollar Cost Points Redeemed,405624.0,2.484503,10.150038,0.0,0.00,0.0,0.00,71.0



Estadísticas descriptivas para VARIABLES CATEGÓRICAS:
--> No tenemos variables categóricas en este df.


→ CUSTOMER LOYALTY HISTORY

In [ ]:
print("Estadísticas descriptivas para VARIABLES NUMÉRICAS:")
display(df_clh.describe().T)
print("\nEstadísticas descriptivas para VARIABLES CATEGÓRICAS:")
# Country / Province / City / Postal Code / Gender / Education / Martial Status / Loyalty Card / Enrollment Type  --> ok
display(df_clh.describe(include = "object").T)

# NOTAS:


Estadísticas descriptivas para VARIABLES NUMÉRICAS:


,count,mean,std,min,25%,50%,75%,max
Loyalty Number,16737.0,549735.880445,258912.132453,100018.00,326603.00,550434.00,772019.00,999986.00
Salary,12499.0,79245.609409,35008.297285,-58486.00,59246.50,73455.00,88517.50,407228.00
CLV,16737.0,7988.896536,6860.982280,1898.01,3980.84,5780.18,8940.58,83325.38
Enrollment Year,16737.0,2015.253211,1.979111,2012.00,2014.00,2015.00,2017.00,2018.00
Enrollment Month,16737.0,6.669116,3.398958,1.00,4.00,7.00,10.00,12.00
Cancellation Year,2067.0,2016.503145,1.380743,2013.00,2016.00,2017.00,2018.00,2018.00
Cancellation Month,2067.0,6.962748,3.455297,1.00,4.00,7.00,10.00,12.00



Estadísticas descriptivas para VARIABLES CATEGÓRICAS:


,count,unique,top,freq
Country,16737,1,Canada,16737
Province,16737,11,Ontario,5404
City,16737,29,Toronto,3351
Postal Code,16737,55,V6E 3D9,911
Gender,16737,2,Female,8410
Education,16737,5,Bachelor,10475
Marital Status,16737,3,Married,9735
Loyalty Card,16737,3,Star,7637
Enrollment Type,16737,2,Standard,15766
